<a href="https://colab.research.google.com/github/sarthakkaushik/Cdiscount-Image-Classification/blob/main/Reading_Tfrecord_File.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Code to mount google drive in case you are loading the data from your google drive
from google.colab import drive
drive.mount('/gdrive')
%cd /gdrive

Mounted at /gdrive
/gdrive


In [2]:
import os 
data_path = '/gdrive/MyDrive/UOH Assignment Dataset/cdiscount'
os.chdir(data_path)
print(os.getcwd())

/gdrive/MyDrive/UOH Assignment Dataset/cdiscount


In [3]:
import os, sys, math, io
import numpy as np
import pandas as pd
import multiprocessing as mp
import bson
import struct

%matplotlib inline
import matplotlib.pyplot as plt

import keras
from keras.preprocessing.image import load_img, img_to_array
import tensorflow as tf

from collections import defaultdict
from tqdm import *

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

from subprocess import check_output
print(check_output(["ls", "../cdiscount"]).decode("utf8"))
# Any results you write to the current directory are saved as output.

categories.csv
category_names.csv
file.h5
sample_submission.csv
test.bson
train.bson
train_example.bson
train_images.csv
train_offsets.csv
train_TFrecords.tfrecords
val_images.csv



In [4]:
#Reading Tfrecord file

data_dir = "../cdiscount/"

train_tfrecord_path = os.path.join(data_dir, "train_TFrecords.tfrecords")


raw_dataset = tf.data.TFRecordDataset(train_tfrecord_path)



In [ ]:
for raw_record in raw_dataset.take(10):
  print(repr(raw_record))

In [ ]:
for raw_record in raw_dataset.take(1):
  example = tf.train.Example()
  example.ParseFromString(raw_record.numpy())
  print(example)

In [11]:
def parse_tfr_element(element):
  #use the same structure as above; it's kinda an outline of the structure we now want to create
  data = {
      'height': tf.io.FixedLenFeature([], tf.int64),
      'width':tf.io.FixedLenFeature([], tf.int64),
      'depth':tf.io.FixedLenFeature([], tf.int64),
      'category_id':tf.io.FixedLenFeature([], tf.int64),      
      'product_id':tf.io.FixedLenFeature([], tf.int64),
      'img_raw' : tf.io.FixedLenFeature([], tf.string)
    }

    
  content = tf.io.parse_single_example(element, data)
  
  height = content['height']
  width = content['width']
  depth = content['depth']
  label = content['category_id']
  product_id = content['product_id']
  img_raw = content['img_raw']
  
  
  #get our 'feature'-- our image -- and reshape it appropriately
  feature = tf.io.decode_raw(img_raw,tf.uint8,fixed_length=97200)
  feature = tf.reshape(feature, shape=[height,width,depth])
  return (feature, label)

To create a dataset out of the parse elements, we simply leverage the tf.data API. We create a TFRecordDataset by pointing it to the TFRecord file on our disk and then apply our previous parsing function to every extracted Example. This returns a dataset:

In [12]:
def get_dataset_small(filename):
  #create the dataset
  dataset = tf.data.TFRecordDataset(train_tfrecord_path, num_parallel_reads=4)
          # = tf.data.TFRecordDataset(train_tfrecord_path, compression_type = 'ZLIB')

  #pass every single feature through our mapping function
  dataset = dataset.map(
      parse_tfr_element
  )
    
  return dataset

We can explore the content of our dataset by taking a single data point:


In [13]:
dataset_small = get_dataset_small(train_tfrecord_path)

# for sample in dataset_small.take(1):
#   print(sample[0].shape)
#   print(sample[1].shape)

In [14]:
type(dataset_small)

tensorflow.python.data.ops.dataset_ops.MapDataset

In [15]:
for data in dataset_small.take(1):
    print(data,type(data))

(<tf.Tensor: shape=(180, 180, 3), dtype=uint8, numpy=
array([[[255, 216, 255],
        [224,   0,  16],
        [ 74,  70,  73],
        ...,
        [117, 118, 119],
        [120, 121, 122],
        [130, 131, 132]],

       [[133, 134, 135],
        [136, 137, 138],
        [146, 147, 148],
        ...,
        [ 11,  28, 241],
        [178, 171,  50],
        [142, 219, 131]],

       [[ 12, 215, 166],
        [215, 124,  36],
        [167,  21,  36],
        ...,
        [175,  21, 197],
        [ 75,  20, 155],
        [246, 105, 107]],

       ...,

       [[  0,   0,   0],
        [  0,   0,   0],
        [  0,   0,   0],
        ...,
        [  0,   0,   0],
        [  0,   0,   0],
        [  0,   0,   0]],

       [[  0,   0,   0],
        [  0,   0,   0],
        [  0,   0,   0],
        ...,
        [  0,   0,   0],
        [  0,   0,   0],
        [  0,   0,   0]],

       [[  0,   0,   0],
        [  0,   0,   0],
        [  0,   0,   0],
        ...,
        [  0,   0,  

FROM THE AUTHOR